In [32]:
from pathlib import Path
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import os
import psycopg2
from flask import Flask, render_template
import pandas as pd

In [33]:
conn = psycopg2.connect(database="flask_df", user="flask_df_user", password="DF4LP1UcZJt3AN4cUW9hrbfp2p4FtWL3", host="dpg-cgtiqjl269vbmeuj26cg-a.oregon-postgres.render.com", port=5432)

# Open a cursor to perform database operations
cur = conn.cursor()

In [34]:
cur.execute('SELECT * FROM movie_successs;')
books = cur.fetchall()
col_name = ['name', 'rating', 'genre', 'year', 'released', 'score', 'votes', 'director', 'writer', 'star', 'country', 'budget', 'gross', 'company', 'runtime', 'cpi', 'ticket']
df_movie = pd.DataFrame(books, columns = col_name)

In [35]:
#Split release data and create a monthly column

df_movie["released"] = df_movie["released"].str.split("\s+\(").str[0]

df_movie["released"] = pd.to_datetime(df_movie['released'])

df_movie ["released"] = df_movie["released"].dt.month_name()

In [45]:
df_movie = df_movie[pd.to_numeric(df_movie['score'], errors='coerce').notnull()]
df_movie = df_movie[pd.to_numeric(df_movie['budget'], errors='coerce').notnull()]
df_movie = df_movie[pd.to_numeric(df_movie['gross'], errors='coerce').notnull()]
df_movie = df_movie[pd.to_numeric(df_movie['votes'], errors='coerce').notnull()]
df_movie['cpi'] = df_movie['cpi'].astype('float')
df_movie['ticket'] = pd.to_numeric(df_movie['ticket'])
df_movie.dtypes


name                object
rating              object
genre               object
year                 int64
released            object
score               object
votes              float64
director            object
writer              object
star                object
country             object
budget             float64
gross              float64
company             object
runtime            float64
cpi                float64
ticket             float64
title_char           int64
sequels              int32
director_number     object
writer_number       object
star_number         object
trixbudget         float64
success              int32
dtype: object

In [37]:
# add column for character count of movie title

df_movie['title_char'] = df_movie['name'].str.len()

In [38]:
# look for sequel indicator and assign value of 1 or 0

df_movie['sequel'] = df_movie['name'].str.extract('(^\d*)')

df_movie['sequels']=df_movie['sequel'].isin([""]).astype(int)

df_movie = df_movie.drop(columns=['sequel'])

In [39]:
# added averaged score values for director, writer and star

df_movie['director_number'] = df_movie.groupby('director')['score'].transform('sum')/df_movie.groupby('director')['score'].transform('count')
df_movie['writer_number'] = df_movie.groupby('writer')['score'].transform('sum')/df_movie.groupby('writer')['score'].transform('count')
df_movie['star_number'] = df_movie.groupby('star')['score'].transform('sum')/df_movie.groupby('star')['score'].transform('count')

In [40]:
# assign 1 and 0 for success or failure
df_movie['trixbudget']= df_movie['gross'] - 3.0*df_movie['budget']
df_movie['success'] = df_movie['trixbudget'].map(lambda x: x>0).astype(int)

In [41]:
# drop unnecessary values

df_movie_clean = df_movie.drop(columns=['name', 'trixbudget', 'gross', 'score', 'votes'])

df_movie_clean = df_movie_clean[df_movie_clean.budget > 15000000]

df_movie_clean = df_movie_clean[df_movie_clean.year > 1985]

df_movie_clean = df_movie_clean.dropna()

In [42]:
# assign values as other for counts under certain values

director_other = df_movie_clean['director'].value_counts() < 15
df_movie_clean.loc[df_movie_clean['director'].isin(director_other.index[director_other]), 'director'] = 'others'

writer_other = df_movie_clean['writer'].value_counts() < 10
df_movie_clean.loc[df_movie_clean['writer'].isin(writer_other.index[writer_other]), 'writer'] = 'others'

star_other = df_movie_clean['star'].value_counts() < 11
df_movie_clean.loc[df_movie_clean['star'].isin(star_other.index[star_other]), 'star'] = 'others'

country_other = df_movie_clean['country'].value_counts() < 25
df_movie_clean.loc[df_movie_clean['country'].isin(country_other.index[country_other]), 'country'] = 'others'

company_other = df_movie_clean['company'].value_counts() < 100
df_movie_clean.loc[df_movie_clean['company'].isin(company_other.index[company_other]), 'company'] = 'others'

rating_other = df_movie_clean['rating'].value_counts() < 100
df_movie_clean.loc[df_movie_clean['rating'].isin(rating_other.index[rating_other]), 'rating'] = 'others'

genre_other = df_movie_clean['genre'].value_counts() < 20
df_movie_clean.loc[df_movie_clean['genre'].isin(genre_other.index[genre_other]), 'rating'] = 'others'

In [46]:
#cpi adjustments:
df_movie_clean['budget'] = df_movie_clean['budget']/df_movie_clean['cpi']
df_movie_clean['ticket'] = df_movie_clean['ticket']/df_movie_clean['cpi']

TypeError: unsupported operand type(s) for /: 'decimal.Decimal' and 'float'

In [24]:
# create dummy for non-integer variables
df_movie_dummies = pd.get_dummies(df_movie_clean)

In [25]:
#split x and y values and create train test split
y = df_movie_dummies["success"]

X = df_movie_dummies.drop(columns=['success'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [26]:
# scale x variables

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)

In [27]:
# create model

nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=20, activation="sigmoid", input_dim=133))
nn_model.add(tf.keras.layers.Dense(units=8, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn_model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=11)


model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/11


ValueError: in user code:

    File "C:\Users\Chris\anaconda3\envs\PythonData\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Chris\anaconda3\envs\PythonData\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Chris\anaconda3\envs\PythonData\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Chris\anaconda3\envs\PythonData\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Chris\anaconda3\envs\PythonData\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Chris\anaconda3\envs\PythonData\lib\site-packages\keras\engine\input_spec.

In [ ]:
df_movie_clean

In [ ]:
df_movie_dummies = pd.get_dummies(df_movie_clean)

y = df_movie_dummies["success"]

X = df_movie_dummies.drop(columns=['success'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)

nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=20, activation="sigmoid", input_dim=133))
nn_model.add(tf.keras.layers.Dense(units=8, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=11)


model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")